### 문제 1-1 : 기본 체인 만들기 - AI 요리사

In [16]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [12]:
prompt = PromptTemplate.from_template('''
    당신은 어떤 재료로든 요리할 수 있는 일류 요리사입니다.
    다음 질문에 답해주세요.
    질문: {input}으로 요리할 수 있는 요리는 무엇이 있나요?(하나만 알려주세요) 그 요리의 레시피도 알려주세요.
''')

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = prompt | llm

try:
    result = chain.invoke({"input": "계란, 밥, 김치"})
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

**계란, 밥, 김치로 요리할 수 있는 요리: 김치밥**

**김치밥 레시피**

**재료:**

*   2컵 분량의 밥 (햇밥 또는 찬밥)
*   1/2컵 분량의 김치 (잘게 썬 김치)
*   2개의 계란
*   1테이블스푼의 참기름
*   소금과 후추

**과정:**

1.  밥을 준비합니다. 햇밥을 사용하는 경우, 밥을 조금 식힌 후 사용합니다. 찬밥을 사용하는 경우, 냉장고에서 꺼내 실온에 30분 정도 두어 부드럽게 합니다.
2.  김치를 잘게 썰어둡니다. 
3.  팬에 참기름을 두르고 썬 김치를 넣고 볶습니다. 김치가 약간 볶아지면 소금과 후추로 간을 합니다. 
4.  김치볶음이 끝나면, 팬에 계란을 깨뜨려 넣고 잘 섞어줍니다. 계란이 반숙 상태가 되면, 준비한 밥을 넣고 모든 재료를 잘 섞어줍니다. 
5.  모든 재료가 섞이면, 소금과 후추로 간을 다시 한번 확인하고 필요하면 추가합니다.
6.  마지막으로 참기름을 조금 더 뿌려주면 완성입니다.

이렇게 간단하게 김치밥을 만들 수 있습니다.


### 문제 1-2 : 2단계 체인 만들기 - 영화 추천 시스템

In [18]:
# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천하는 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 영화의 제목, 영화의 줄거리(3문장), 영화의 정보(감독, 등장인물)등을 알려주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )
    
    response = chain2.invoke({"genre": "액션"})
    print(response) 
except Exception as e:
    print(f"오류 발생: {e}")

**영화의 제목:** 다이하드

**영화의 줄거리:** 크리스마스 파티를 즐기던 뉴욕 경찰 존 맥클레인은 테러리스트들의 나카토미 빌딩 침입으로 인해 홀로 빌딩에 갇히게 됩니다. 테러리스트들은 빌딩에 있는 사람들을 인질로 잡고, 존 맥클레인은 자신의 가족과 동료들을 구하기 위해 테러리스트들과 치열한 대결을 벌입니다. 존 맥클레인은 자신의 지혜와 용기를 발휘하여 테러리스트들을 하나씩 무찌르며 빌딩을 탈출하기 위해 노력합니다.

**영화의 정보:** 

*   감독: 존 맥티어난
*   등장인물: 브루스 윌리스 (존 맥클레인), 알란 릭맨 (한스 그루버), 보니 베델리아 (홀리 맥클레인)


### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용

In [17]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "경기 의정부시에서 6중 추돌사고 낸 포르쉐 운전자가 마약 간이 검사에서 양성 반응을 보인 것으로 확인됐다.",
        "keywords": "의정부시, 추돌사고, 마약"
    }
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])


# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."})
print(result.content)

키워드: 제미나이, 구글, AI
